# [실습1] 결정론적 시뮬레이션 환경을 활용한 Q-Learning 실습

이번 실습에서는 간단한 결정론적 시뮬레이션 환경을 활용하여 Q-Learning을 직접 구현해보도록 하겠습니다.

여기서 결정론적인 환경이란, 에이전트가 특정한 행동을 취했을 때 그에 따른 다음 상태와 보상이 **하나로 결정되어** 항상 동일한 환경을 의미합니다.

### 라이브러리 불러오기

필요한 라이브러리를 불러옵니다.

In [ ]:
# 필요한 라이브러리를 불러옵니다.
import gymnasium as gym
import numpy as np
import pandas as pd

# 학습을 완료한 에이전트를 시각화하는데 필요한 라이브러리를 불러옵니다
from IPython import display # IPython 라이브러리에 있는 display 모듈을 사용합니다.
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

## 1. Taxi-v3 시뮬레이션 환경

랜덤한 위치에 택시, 승객, 목적지의 위치가 정해지면 택시를 이용하여 승객을 목적지까지 이동시키는 간단한 결정론적 시뮬레이션 환경입니다.

![taxi](./images/taxi_v3.gif)

Environment는 다음과 같이 정의됩니다.

- 가능한 모든 상태 집합 S 의 원소의 수 = 택시의 위치 25개 x 승객의 위치 5개 (4색 타일 + 택시 안) x 목적지 위치 4개 (4색 타일) = 500개
- 가능한 행동 집합 A = {상, 하, 좌, 우, 태우기, 내리기}
- 전이 함수 P: 벽에 가로막히지 않는 이상 행동을 그대로 수행할 확률 100% (즉, 확률적이지 않음)
- 보상 함수 R: 목적지에 승객을 내리면 +20, 그 외에는 -1, 적절하지 않은 태우기 및 내리기 행동을 하면 -10
- 감가율 γ (gamma) = 0.95 (Taxi-v3 환경의 자체 설정이 아니라, 임의로 설정한 값입니다)

예시 화면에서는 택시의 가능한 위치가 45개인 것 처럼 보이나, 짝수번째 열은 실제로는 존재하지 않는 영역이며, 벽은 두께 0의 벽으로 표현됩니다. 따라서 가능한 택시의 위치는 25개입니다.

Taxi-v3 환경을 불러옵니다.

In [ ]:
# 'Taxi-v3' 환경을 생성합니다.
env = gym.make('Taxi-v3')

학습 과정에서 생성할 Q Table의 크기를 확인합니다.

In [ ]:
# 학습 과정에서 생성할 Q-Table의 크기를 확인합니다. Envirionment의 상태(State)의 개수는 500개이며, Action의 개수는 6개입니다.
print([env.observation_space.n, env.action_space.n])

Q Table은 각 State마다 가치를 부여 해야 하므로, 결국 Q Table의 크기는 상태 집합의 크기와 같은 500입니다.
- 가능한 모든 상태 집합 S의 크기: 택시의 위치 25개 x 승객의 위치 5개 (4색 타일 or 택시 안) x 목적지 위치 4개 (4색 타일) = 500

## 2. 학습 환경 구축

학습 파라미터를 설정합니다.

In [ ]:
# 학습 파라미터를 설정합니다.
learning_rate = 0.1 # 학습률 eta
gamma = 0.95 # 감가율
num_episodes = 5000 # 에피소드 수
max_steps = 100 # 각 에피소드 별 최대 스텝 수

아래 코드는 1 episode 만큼 학습을 진행하는 코드입니다. 이 코드로 학습한 Q_table은 차후 충분한 학습을 거친 Q_table과 비교하여 차이를 확인해 볼 것입니다.

In [ ]:
# 임의의 Q-Table을 생성합니다. 모든 값을 랜덤하게 초기화합니다.
Q_1ep = np.random.rand(env.observation_space.n, env.action_space.n)

state = env.reset()[0] # 환경을 초기화하고 초기 상태를 가져옵니다.

done = False
for i in range(max_steps):
    # 현재 상태(state)를 바탕으로 Q-Table에서 행동을 결정합니다. Q 값이 가장 높은 행동을 선택합니다.
    action = np.argmax(Q_1ep[state,:])

    # 환경에서 새로운 상태와 보상을 얻습니다.
    next_state, reward, done, _, _ = env.step(action)
    
    # 오차 = 보상 + 감가율 * (다음 상태에서의 최대 Q 값) - (현재 상태에서의 Q 값)
    error = reward + gamma * np.max(Q_1ep[next_state,:]) - Q_1ep[state,action]

    # Q[S,A] ← Q[S,A] + 학습률 * 오차
    Q_1ep[state,action] = Q_1ep[state,action] + learning_rate * error
    state = next_state
    if done == True:
        break

### [TODO] 학습을 진행하는 함수를 정의합니다.
- 위의 예시 코드를 참고하여, 학습을 진행하는 함수를 정의합니다.

In [ ]:
# 환경과 학습 파라미터를 입력 받으면 Q-Table을 학습하여 리턴하는 함수를 정의합니다.
def Q_learning(env, gamma, learning_rate, num_episodes, max_steps, Q_table = None):
    
    # 파라미터로 Q_table이 주어질 경우 그를 바탕으로 학습을 진행합니다.
    if Q_table is None: 
        Q = np.ones([env.observation_space.n, env.action_space.n])
    else:
        Q = Q_table
    
    # 에피소드별 총 보상과 스텝을 저장할 리스트를 생성합니다.
    rList = []
    for i in range(num_episodes):
    # 환경을 초기화하고 첫 번째 새로운 관찰을 얻습니다.
        state = env.reset()[0]
        rAll = 0
        done = False
        # Q-Table 학습 알고리즘
        for j in range(max_steps):

            # TODO: 현재 상태(state)를 바탕으로 Q-Table에서 행동을 결정합니다.
            # Hint: Q[S,:]는 현재 상태가 S일 때 각 행동에 대한 Q 값을 담고 있는 list를 나타냅니다.
            action = None
            
            # 환경에서 새로운 상태와 보상을 얻습니다.
            next_state, reward, done, _, _ = env.step(action)
            
            # TODO: 오차를 계산합니다. (gamma = 감가율)
            # Hint: 오차 = 보상 + 감가율 * (다음 상태에서의 최대 Q 값) - (현재 상태에서의 Q 값)
            error = None

            # TODO: 오차를 기반으로 Q-Table을 업데이트합니다.
            # Hint: Q[S,A] ← Q[S,A] + 학습률 * 오차
            Q[state,action] = None

            rAll += reward
            state = next_state
            if done == True:
                break
        
        # 에피소드별 총 보상을 리스트에 저장합니다.
        rList.append(rAll)      
    
    # 학습된 Q-Table을 리턴합니다.
    return Q, rList

## 3. 학습 진행

학습을 진행합니다.

In [ ]:
Q, rList = Q_learning(env, gamma, learning_rate, num_episodes, max_steps)

rList 리스트에는 각 에피소드에서 얻은 reward가 저장됩니다. 이를 통해 학습이 진행됨에 따라 얻는 reward가 어떻게 변화하는지 확인할 수 있습니다.

rList를 시각화 해봅시다.

In [ ]:
# rList를 시각화합니다.
plt.plot(rList)
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()

대부분의 경우 Episode 2000 이상부터 reward가 0 이상이 되는 것을 확인할 수 있습니다. 해당 구간만 잘라서 시각화 해보겠습니다.

In [ ]:
# Episode 2000부터 rList를 시각화합니다.
plt.plot(rList[2000:])
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()

## 4. 결과 시각화

결과를 시각화하기 위해, 입력 받은 Q Table을 적용한 Agent를 정해진 환경에서 시뮬레이팅 하는 함수를 정의합니다.

In [ ]:
def simulate_Q(Q):
    test_seed = 42
    # 테스트 및 시각화를 위해 시각화 가능한 새로운 환경을 생성합니다.
    env_test = gym.make('Taxi-v3', render_mode="rgb_array")
    state = env_test.reset(seed = test_seed)[0]

    # 초기 상태 렌더링
    screen = env_test.render()
    screen_queue = [screen]
    
    # 최대 10 step까지 시각화합니다.
    for i in range(100):
        action = np.argmax(Q[state,:])
        state, reward, done, truncated, info = env_test.step(action)
        screen = env_test.render()
        screen_queue.append(screen)
        if done:
            print("{} timesteps 만에 종료되었습니다.".format(i+1))
            break

    # env_test를 닫습니다.
    env_test.close()

    # 시뮬레이션 결과를 시각화하기 위한 정보를 리턴합니다.
    return screen_queue   

1 Episode만 학습한 Q Table을 이용하여 시뮬레이션을 진행합니다.

In [ ]:
screen_queue_1ep = simulate_Q(Q_1ep)

1 Episode만 학습한 Q Table을 적용한 Agent의 시뮬레이팅 결과를 시각화합니다.

In [ ]:
fig = plt.figure()
ims = []
for i in range(len(screen_queue_1ep)):
    im = plt.imshow(screen_queue_1ep[i], animated=True)
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=300, blit=True,
                                repeat_delay=1000)
HTML(ani.to_jshtml())

다음으로, 학습 완료한 Q Table을 적용한 Agent를 시뮬레이팅 하고, 시각화 해봅시다.

In [ ]:
screen_queue = simulate_Q(Q)

# Agent의 시뮬레이팅 결과를 시각화합니다.

fig = plt.figure()
ims = []
for i in range(len(screen_queue)):
    im = plt.imshow(screen_queue[i], animated=True)
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=300, blit=True,
                                repeat_delay=4000)
HTML(ani.to_jshtml())

In [ ]:
# 애니메이션을 gif로 저장합니다.
ani.save('taxi-v3.gif', writer='Pillow', fps=2)

### 결과 제출

- 학습을 완료한 Q Table을 제출해주세요. 채점 환경에서 해당 Q Table을 적용한 Agent가 목적을 달성하면 성공입니다.

In [ ]:
# Q-Table을 csv 파일로 변환하여 제출합니다.
df = pd.DataFrame(Q)
df.to_csv('taxi-v3.csv', index=False, header=False)

### 추가 선택 과제

- 채점에 포함되지는 않습니다.
- 학습 파라미터를 변경해서 더 적은 episode 수만 가지고 동일하거나 더 높은 성능을 내는 Agent를 학습시켜보세요.